## Data Engineering about Movies 🚀
I'm excited to share the progress in my data engineering studies! 🚀
In this project we are preparing data about movies and serving it to a scalable Storage called Big Query . The data is about the likability , length of movies .


In this repository, I explore one of the many possible options for implementing an ETL pipeline.

- I'm using two types of data sources, a local one represented by a MySQL database, and a remote one represented by a Public API. 🏛️🌐
- In the two working notebooks (API_ETL.ipynb, etl.ipynb), I explore two ways of transforming data. The first method uses a Pandas transformation in a local environment, and the second involves executing a data extraction using an API with the necessary transformations to add business context to the data making it ready for analysis. 📊🔄
- Finally, using Python libraries for Google Cloud Platform, I store these analysis-ready data in BigQuery, completing the ETL process. 🐍🔍📊

![ETL DIAGRAM](./images/Movies_2024.png)

## Required Python Libraries
Firstly we are going to install a couple of python libraries that will help us to execute the Extract Transform and Load Process . What we do is that we list all the Libraries in a file called 'requirements.txt'
the libraries include.

To install the above libraries , we run a python command

In [ ]:
pip install -r requirements.txt

# Extract Transform Load Overview 

## Extract 
We are going to extract data from MYSQL database , We do this to get data from the production environment into an analytical environment we do this ensure optimized performance of the production database. We shall be extracting our data from an SQL database in this project

## Transform
This step is optional but this is the step where by we get to add columns , change data types of data columns . This is done according to business rules.

## Load
This is the step where we load our data into analytical environments , there are different tools that are used to achieve this but we are going to use Google Big query(Scalable Storage) in this project.

## Connect to Python using MYSQL

What we do is that we put credentials that allow access to an online mysql database in file called credentials.py which you won't see here because i have instructed git to ignore it . I do this as a security measure not to reveal the keys to the production database.


Next i write code for connection to the mysql database . I only write this code here once to avoid repetition of code.



In [1]:
import mysql.connector
# import create_customers
import credentials
# Replace these variables with your actual database credentials


db_config = {
    'user': credentials.user[0],
    'password': credentials.password[0],
    'host': credentials.host[0],
    'database': credentials.database[0],
}


# Establish a connection to the database
connection = mysql.connector.connect(**db_config)
try:
    # Establish a connection to the database
    connection = mysql.connector.connect(**db_config)
    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f"Connected to MySQL Server version {db_info}")

except mysql.connector.Error as e:
    print(f"Error connecting to MySQL: {e}")

Connected to MySQL Server version 5.7.23-23


## Pandas Data Types 
The next thing that we are doing is to do is the check the data types of the columns in our data using pandas data types , we are doing this inorder to check the types of data that we do have in our data such that we faciliate further transformations like mathematical operations in accordance to business rules. One has to be sure of the types of data that they are dealing with.

In [13]:
import pandas as pd

query = "select year " \
        ", title " \
        ", genre "  \
        "from `oscarval_sql_course`.`imdb_movies` " \
        "limit 7"

df = pd.read_sql(query,connection)

print(df.head())

print(df.dtypes)



C:\Users\Jacob\AppData\Local\Temp\ipykernel_30496\1593940848.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,connection)


   year                                              title  \
0  1906                        The Story of the Kelly Gang   
1  1911                                     Den sorte drøm   
2  1912                                          Cleopatra   
3  1911                                          L'Inferno   
4  1912  From the Manger to the Cross; or, Jesus of Naz...   

                       genre  
0    Biography, Crime, Drama  
1                      Drama  
2             Drama, History  
3  Adventure, Drama, Fantasy  
4           Biography, Drama  
year      int64
title    object
genre    object
dtype: object


## Pandas filters

This enables us to filter out information that is required to fulfill the business rule , for example in this instance we need to see the movies that were established in 2005



In [34]:
query = "select year " \
        ",  title " \
        ",  genre " \
        ",  avg_vote " \
        "from `oscarval_sql_course`.`imdb_movies` " \
        "where year between 2005 and 2006"
        

df = pd.read_sql(query, connection)

yr_2005 = df['year'] == 2005

print(df[yr_2005].head()) 


C:\Users\Jacob\AppData\Local\Temp\ipykernel_42616\2153758935.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


   year                                         title  \
0  2005                             The Naked Monster   
1  2005                                   What Is It?   
2  2005                                Fantastic Four   
3  2005                                  Corpse Bride   
4  2005  Star Wars: Episode III - Revenge of the Sith   

                        genre  avg_vote  
0      Comedy, Horror, Sci-Fi       5.6  
1                       Drama       5.6  
2  Action, Adventure, Fantasy       5.7  
3    Animation, Drama, Family       7.3  
4  Action, Adventure, Fantasy       7.5  


## Pandas Export to CSV
What we are now doing is that we are extracting the data from mysql into pandas then to csv , We do this to prepare our data to be ingested in the Cloud warehouse Big query. In this cell we want to see the movies that were released in 2005 being stored in csv waiting to be moved to a scalable storage like Google Big Query.

In [14]:
import os

cur_path = os.getcwd()

file = 'movies.csv'

file_path = os.path.join(cur_path,'data_files',file)

print(file_path)
query = "select year "\
        ",  title "\
        ",  genre "\
        ",  avg_vote "\
        "from `oscarval_sql_course`.`imdb_movies` "\
        "where year between 2005 and 2006" 

df = pd.read_sql(query, connection)

yr_2005 = df['year'] == 2005

df[yr_2005].to_csv(file_path)

c:\Users\Jacob\code\portifolio_projects\Extract_Load_Transform\data_files\movies.csv


C:\Users\Jacob\AppData\Local\Temp\ipykernel_30496\736300270.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


## Pandas new Column using SQL

Here we are going to grade the movies and see if they were good movies or bad movies according the average votes that they received

In [17]:
import os
import pandas as pd

cur_path = os.getcwd()

file = 'movie_rating.csv'

file_path = os.path.join(cur_path,'data_files',file)

print(file_path)
query = "select year "\
        ",  title "\
        ",  genre "\
        ",  avg_vote "\
        ",  case "\
        " when avg_vote < 3 then 'bad' "\
        " WHEN avg_vote BETWEEN 3 AND 6 THEN 'okay' "\
        " when avg_vote >= 6 then 'good' "\
        " end as movie_rating " \
        " from `oscarval_sql_course`.`imdb_movies` "\
        "where year between 2005 and 2006" 

df = pd.read_sql(query, connection)

yr_2005 = df['year'] == 2005

df[yr_2005].to_csv(file_path)

c:\Users\Jacob\code\portifolio_projects\Extract_Load_Transform\data_files\movie_rating.csv


C:\Users\Jacob\AppData\Local\Temp\ipykernel_30496\3021959094.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


## Pandas new Column with Python Function

Here we are going to look at length of the movies and categorize them and see if they were really short , average and really long .Essentially here we are looking to see the watchability of the movie . Some people like long movies others like short movies 

In [34]:
import os
import pandas as pd

cur_path = os.getcwd()

file = 'watchability.csv'

file_path = os.path.join(cur_path,'data_files',file)

print(file_path)
query = "select year "\
        ",  title "\
        ",  genre "\
        ",  avg_vote "\
        ",  case "\
        " when avg_vote < 3 then 'bad' "\
        " WHEN avg_vote BETWEEN 3 AND 6 THEN 'okay' "\
        " when avg_vote >= 6 then 'good' "\
        " end as movie_rating " \
        ", duration " \
        " from `oscarval_sql_course`.`imdb_movies` "\
        "where year between 2005 and 2006" 

# create duration label function
def movie_duration(d):
    if d < 60:
        return 'short movie'
    elif d > 60 and d < 90:
        return 'avg length movie'
    elif d > 90 and d < 5000:
        return 'really long movie'
    else:
        return 'no data'
    
df = pd.read_sql(query, connection)

df['watchability'] = df['duration'].apply(movie_duration)

df.to_csv(file_path)

c:\Users\Jacob\code\portifolio_projects\Extract_Load_Transform\data_files\watchability.csv


C:\Users\Jacob\AppData\Local\Temp\ipykernel_30496\4284993103.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


## Loading Data to Big Query
We are now loading data to big query using python , this is the start of the final step in the ETL process .We are looking at moving the movie_rating , movies and watchability data of the movies in the Big query.

In [31]:
from google.cloud import bigquery
import os

client = bigquery.Client(project='charming-autumn-407214')
target_table_1 = 'charming-autumn-407214.sample_dataset.movies'

job_config = bigquery.LoadJobConfig(
    skip_leading_rows = 1,
    source_format = bigquery.SourceFormat.CSV,
    autodetect=True
)

# file vars
cur_path = os.getcwd()
file = 'movies.csv'
file_path = os.path.join(cur_path,'data_files',file)

with open(file_path, 'rb') as source_file:
    load_job   = client.load_table_from_file(
        source_file,
        target_table_1,
        job_config=job_config

    )

load_job.result()

destination_table = client.get_table(target_table_1)

print(f"You have {destination_table.num_rows} rows in your table ")



You have 1806 rows in your table 


In [2]:
#watchability
from google.cloud import bigquery
import os

client = bigquery.Client(project='charming-autumn-407214')
target_table_2 = 'charming-autumn-407214.sample_dataset.watchability'

job_config = bigquery.LoadJobConfig(
    skip_leading_rows = 1,
    source_format = bigquery.SourceFormat.CSV,
    autodetect=True
)

# file vars
cur_path = os.getcwd()
file = 'watchability.csv'
file_path = os.path.join(cur_path,'data_files',file)

with open(file_path, 'rb') as source_file:
    load_job   = client.load_table_from_file(
        source_file,
        target_table_2,
        job_config=job_config

    )

load_job.result()

destination_table = client.get_table(target_table_2)

print(f"You have {destination_table.num_rows} rows in your table ")

You have 3851 rows in your table 


In [2]:
#movie_rating
from google.cloud import bigquery
import os

client = bigquery.Client(project='charming-autumn-407214')
target_table_2 = 'charming-autumn-407214.sample_dataset.movie_rating'

job_config = bigquery.LoadJobConfig(
    skip_leading_rows = 1,
    source_format = bigquery.SourceFormat.CSV,
    autodetect=True
)

# file vars
cur_path = os.getcwd()
file = 'movie_rating.csv'
file_path = os.path.join(cur_path,'data_files',file)

with open(file_path, 'rb') as source_file:
    load_job   = client.load_table_from_file(
        source_file,
        target_table_2,
        job_config=job_config

    )
load_job.result()

destination_table = client.get_table(target_table_2)

print(f"You have {destination_table.num_rows} rows in your table ")

You have 5657 rows in your table 
